In [ ]:
import pickle

with open("AndroidModules.pkl", "rb") as target:
    third_party_calls, third_party_total_call, detected_libs_infos = pickle.load(target)

In [ ]:
s = 0
cs = 0 
ls = 0
for lib in third_party_calls:
    ls += 1
    for c in third_party_calls[lib]:
        cs += 1
        for m in third_party_calls[lib][c]:
            s += third_party_calls[lib][c][m]["count"]
print(s)
print(cs)
print(ls)

In [ ]:
third_party_total_call

In [ ]:
# Check if lib doc is exists
import requests
import re
from bs4 import BeautifulSoup

BASE_URL = "https://javadoc.io"
java_doc_status = {"exist": [], "not_exist":[]}
for lib in third_party_total_call:
    lib_name = lib[1:].replace("/", ".")
    url = "{}/doc/{}".format(BASE_URL, lib_name)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    if soup.find(text=re.compile('has published 0 artifact\(s\) with total 0 version\(s\)')):
        java_doc_status["not_exist"].append(lib)
    else:
        java_doc_status["exist"].append(lib)

In [ ]:
#Get Artifact Names from Valid Libs
BASE_URL = "https://javadoc.io"
lib_artifacts = {}
for lib in java_doc_status["exist"]:
    lib_name = lib[1:].replace("/", ".")
    url = "{}/doc/{}".format(BASE_URL, lib_name)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    lib_artifacts[lib] = []
    for artifact in soup.find("table", {"class": "table table-striped"}).find("tbody").find_all("tr"):
        name = artifact.find("td")
        lib_artifacts[lib].append(name.text)

In [ ]:
possible_match = {}
for lib in java_doc_status["exist"]:
    possible_match[lib] = set()
    for lib_class in third_party_calls[lib]:
        parts = lib_class.split()[-1].split("/")[1:]
        for p in parts:
            for artifact in lib_artifacts[lib]:
                if p in artifact:
                    possible_match[lib].add(artifact)

In [ ]:
print(len(java_doc_status["exist"]), len(java_doc_status["not_exist"]))

In [ ]:
s = 0
cs = 0 
ls = 0
problemcs = 0
for lib in java_doc_status["exist"]:
    ls += 1
    for c in third_party_calls[lib]:
        cs += 1
        n = c.split("/")[-1]
        if n.lower() != n:
            pass
            #print(lib, ",", c)
        else:
            problemcs += 1
            print(lib, ",", c)
        for m in third_party_calls[lib][c]:
            s += third_party_calls[lib][c][m]["count"]
print(s)
print(cs, problemcs)
print(ls)

In [ ]:
for key in possible_match:
    print(key)
    for a in possible_match[key]:
        print(a)
    print()

In [ ]:
len(possible_match)

In [ ]:
downloaded_libs = []
for lib in possible_match:
    for artifact in possible_match[lib]:
        lib_name = lib[1:].replace("/", ".")
        l = {"groupid" : lib_name,
              "artefactid": artifact}
        downloaded_libs.append(l)

In [ ]:
import json
with open('third_parties.json', 'w') as json_file: 
    json.dump(downloaded_libs, json_file) 

In [ ]:
import subprocess
with open('scrape_javadocs.txt', 'w') as f:
    process = subprocess.Popen(['python',
                                '/home/huseyinalecakir/NLG/codes/scripts/scrape_javadocs.py',
                                '/home/huseyinalecakir/NLG/codes/notebooks/ExternalDataProcess/JavaDocs',
                               '/home/huseyinalecakir/NLG/codes/notebooks/ExternalDataProcess/third_parties.json'], stdout=f, stderr=f)

In [ ]:
with open('parse_javadocs.txt', 'w') as f:
    process = subprocess.Popen(['python', 
                                '/home/huseyinalecakir/NLG/codes/scripts/parse_javadocs.py'], stdout=f, stderr=f)

In [ ]:
import pandas as pd
classes = pd.read_csv("classes.csv") 
methods = pd.read_csv("methods.csv") 

In [ ]:
## Classses found in documentation

classes_found_in_documentation = {}
for lib in possible_match:
    lib_name = lib[1:].replace("/", ".")
    for lib_class in third_party_calls[lib]:
        parts = lib_class.split()[-1].split("/")[1:]
        for idx, row in classes.iterrows():
            if row[1].lower() == lib_name.lower():
                if parts[-1].replace(";", "").lower() == row[4].split()[-1].lower():
                    print(lib_class, row[4])
                    if lib not in classes_found_in_documentation:
                        classes_found_in_documentation[lib] = set()
                    classes_found_in_documentation[lib].add(lib_class)
                    print(row[5])
                    print("\n\n")

In [ ]:
## Methods found in documentation

for lib in possible_match:
    lib_name = lib[1:].replace("/", ".")
    for idx, row in methods.iterrows():
        if row[1].lower() == lib_name.lower():
            for lib_class in third_party_calls[lib]:
                parts = lib_class.split()[-1].split("/")[1:]
                if parts[-1].replace(";", "").lower() == row[4].split()[-1].lower():
                    for class_method in third_party_calls[lib][lib_class]:
                        if class_method.split()[-1].split("(")[0].lower() == row[5].split("(")[0].split()[-1].lower():
                            print(class_method, row[5])
                            print(row[6])
                            print("\n\n")

In [20]:
## Classses found in documentation

classes_found_in_documentation = {}
for lib in third_party_total_call:
    lib_name = lib[1:].replace("/", ".")
    for lib_class in third_party_calls[lib]:
        parts = lib_class.split()[-1].split("/")[1:]
        for idx, row in classes.iterrows():
            if row[1].lower() == lib_name.lower():
                if parts[-1].replace(";", "").lower() == row[4].split()[-1].lower():
                    print(lib_class, row[4])
                    if lib not in classes_found_in_documentation:
                        classes_found_in_documentation[lib] = set()
                    classes_found_in_documentation[lib].add(lib_class)
                    print(row[5])
                    print("\n\n")

.class public Lcom/mopub/mobileads/AdViewController; Class AdViewController
nan



.class public Lcom/mopub/mobileads/AdViewController; Class AdViewController
nan



.class public Lcom/mopub/mobileads/AdUrlGenerator; Class AdUrlGenerator
nan



.class public Lcom/mopub/mobileads/AdUrlGenerator; Class AdUrlGenerator
nan



.class public Lcom/mopub/common/ClientMetadata; Class ClientMetadata
Singleton that caches Client objects so they will be available to background threads.



.class public Lcom/mopub/common/ClientMetadata; Class ClientMetadata
Singleton that caches Client objects so they will be available to background threads.



.class public Lcom/mopub/common/LocationService; Class LocationService
nan



.class public Lcom/mopub/common/LocationService; Class LocationService
nan



.class public Lcom/mopub/common/util/DeviceUtils; Class DeviceUtils
nan



.class public Lcom/mopub/common/util/DeviceUtils; Class DeviceUtils
nan



.class public Lcom/tune/TuneParameters; Class TunePara

In [21]:
classes.head()

,Unnamed: 0,groupid,artefactid,api,class,description
0,0,com.newrelic.telemetry,opentelemetry-exporters-newrelic,com.newrelic.telemetry.opentelemetry.export,Class AttributeNames,Names of frequently used / common attribute keys
1,1,com.newrelic.telemetry,opentelemetry-exporters-newrelic,com.newrelic.telemetry.opentelemetry.export,Class AttributesSupport,NaN
2,2,com.newrelic.telemetry,opentelemetry-exporters-newrelic,com.newrelic.telemetry.opentelemetry.export,Class DeltaDoubleCounter,NaN
3,3,com.newrelic.telemetry,opentelemetry-exporters-newrelic,com.newrelic.telemetry.opentelemetry.export,Class DeltaLongCounter,NaN
4,4,com.newrelic.telemetry,opentelemetry-exporters-newrelic,com.newrelic.telemetry.opentelemetry.export,Class MetricPointAdapter,NaN


In [22]:
## Methods found in documentation

for lib in third_party_total_call:
    lib_name = lib[1:].replace("/", ".")
    for idx, row in methods.iterrows():
        if row[1].lower() == lib_name.lower():
            for lib_class in third_party_calls[lib]:
                parts = lib_class.split()[-1].split("/")[1:]
                if parts[-1].replace(";", "").lower() == row[4].split()[-1].lower():
                    for class_method in third_party_calls[lib][lib_class]:
                        if class_method.split()[-1].split("(")[0].lower() == row[5].split("(")[0].split()[-1].lower():
                            print(class_method, row[5])
                            print(row[6])
                            print("\n\n")

.method public static isNetworkAvailable(Landroid/content/Context;)Z public static boolean isNetworkAvailable(@Nullable android.content.Context context)
nan



.method public static getLastKnownLocation(Landroid/content/Context;ILcom/mopub/common/LocationService$LocationAwareness;)Landroid/location/Location; @Nullablepublic static android.location.Location getLastKnownLocation(@NonNull android.content.Context context, int locationPrecision, @NonNull MoPub.LocationAwareness locationAwareness)
Returns the last known location of the device using its GPS and network location providers. This only checks Android location providers as often as MoPub.getMinimumLocationRefreshTimeMillis() says to, in milliseconds. May be null if: Location permissions are not requested in the Android manifest file The location providers don't exist Location awareness is disabled in the parent MoPubView 



.method public static getLastKnownLocation(Landroid/content/Context;ILcom/mopub/common/MoPub$LocationAwaren